#### INFO


***SETUP***

1. *Runtime ->  Change Runtime Type -> Python 3 + GPU*
2. *Fill in `GDRIVE_DIR` in [code #5] (name of the folder in gdrive that contains your code and data)*

***RUNNING THE SCRIPTS***
1. Mount the drive before the first run. Also, sometimes so you may need to re-mount the drive ([code #6])
2. Tensorboard - should be enough to run the script once; however restart should work OK ([code #7])
3. Training - when re-running the training, you **do not** have to run any of the preceding scripts

***ABOUT***

- *[code #1] Install dependencies for google authentification*
- *[code #2] Allow collaboratory to read from & write into your google drive"*
- *[code #3] Download tensorboard accesories*
- *[code #4] Setup paths for tensorboard and your script.*
- *[code #5] Declares the function that will be used to run your tensorflow script.*
- *[code #6] Mounts the google drive to current FS as ./drive.*
- *[code #7] Fires up tensorboard.*
- *[code #8] VM reset script - Kills all processes. Sometimes helps, once the GPU tensorflow stops working, drive fails etc. **! RESETS AUTH !** *
- *[code #9] Sample call to run function that actually runs your script.*


***FINAL NOTES***

Most of this script was copy-pasted from https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d

The Tensorflow part is taken from https://github.com/mixuala/colab_utils.git + Special thanks to Štěpán Hojdar.

Feel free to improve this script - just send me suggestions / bug reports / improved scripts and I will upload it here.

Created by Vladislav Vancák & Petr Houška.

#### SETUP

In [1]:
# DRIVE & AUTH DEPENDENCIES
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 16712 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0

Setting up libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Setting up unattended-upgrades (0.98ubuntu1.1) ...

Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) 

In [2]:
# TENSORBOARD DEPENDENCIES
!git clone https://github.com/mixuala/colab_utils.git

Cloning into 'colab_utils'...
remote: Counting objects: 216, done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 216 (delta 15), reused 40 (delta 12), pack-reused 171
Receiving objects: 100% (216/216), 60.19 KiB | 6.69 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [3]:
# GOOGLE AUTH
from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


#### CONFIG

In [0]:
# SCRIPT PROPERTIES
import os

GDRIVE_DIR="DL_fashion_masks"

ROOT = %pwd
SCRIPT_PATH = os.path.join(ROOT, 'drive', GDRIVE_DIR)
LOG_DIR = os.path.join(SCRIPT_PATH, 'logs')

In [0]:
# RUNNER FUNCTION
import tensorflow as tf
def run_script(name, parameters):
  SCRIPT_FILE= name
  SCRIPT_ARGS= parameters

  os.chdir(SCRIPT_PATH)

  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    print('[WARN]: GPU device not found. Running in CPU mode')
  else:
    print('[INFO]: Found GPU at: {}'.format(device_name))  
  print('<<= <=> =>>')
  
  %run $SCRIPT_FILE $SCRIPT_ARGS

#### GOOGLE DRIVE MOUNT & TENSORBOARD

In [13]:
# MOUNT DRIVE / CLEAR CACHE
import os
os.chdir(ROOT)
  
if os.path.exists('drive'):
  !google-drive-ocamlfuse -cc

else:
  !mkdir -p drive
  !google-drive-ocamlfuse drive

Clearing cache...done


In [16]:
# TENSORBOARD
import os
import sys
import colab_utils.tboard
  
os.chdir(ROOT)
colab_utils.tboard.launch_tensorboard( bin_dir=ROOT, log_dir=LOG_DIR)

ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= http://c22dfae4.ngrok.io


'http://c22dfae4.ngrok.io'

#### UTILS

In [0]:
# RESETS VM
!kill -9 -1 

#### RUN

In [15]:
run_script(
    "fashion_masks.py",
    "\
    --threads 4\
    --epochs 20\
    --batch_size 256\
    --logname default\
    --learning_rate 0.002 --learning_rate_final 0.0001\
    --cnn_c C-64-3-1-same,C-64-3-1-same,M-3-2,C-128-3-1-same,C-128-3-1-same,M-3-2\
    --cnn_m C-128-3-1-same,C-128-3-1-same,F,R-1024,D\
    --cnn_l F,R-500,D\
    "
)

[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
i|acc:0.8226|iou:0.7022
i|acc:0.8562|iou:0.7527
i|acc:0.8680|iou:0.7752
i|acc:0.8764|iou:0.7877
i|acc:0.8832|iou:0.7956
i|acc:0.8806|iou:0.7977
i|acc:0.8860|iou:0.8037
i|acc:0.8918|iou:0.8128
i|acc:0.8962|iou:0.8193
i|acc:0.8936|iou:0.8152
i|acc:0.8990|iou:0.8241
i|acc:0.8972|iou:0.8233
i|acc:0.8980|iou:0.8250
i|acc:0.9012|iou:0.8286
i|acc:0.8968|iou:0.8245
i|acc:0.9022|iou:0.8309
i|acc:0.9020|iou:0.8311
i|acc:0.9012|iou:0.8306
i|acc:0.9028|iou:0.8321
i|acc:0.9042|iou:0.8339
